<a href="https://colab.research.google.com/github/lazy-wolf/TME_6016/blob/main/Assignment_1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [1]:
import torch
from torchvision.datasets import CIFAR100
import time
import itertools
from sklearn.metrics import accuracy_score

**Importing Dataset**

In [2]:
dset_train = CIFAR100(root='.', download=True, train=True)
dset_test = CIFAR100(root='.', train=False)

100%|██████████| 169001437/169001437 [00:02<00:00, 83814336.03it/s] 


Extracting ./cifar-100-python.tar.gz to .


**Data Preprocessing**

In [3]:
x_train = torch.tensor(dset_train.data, dtype=torch.float32).div_(255)
y_train = torch.tensor(dset_train.targets, dtype=torch.int64)

In [4]:
x_train = torch.flatten(x_train,start_dim=1) #Flattening the 3 dimension image into a single dimension for easier calculation

In [5]:
x_train.shape

torch.Size([50000, 3072])

In [6]:
x_test = torch.tensor(dset_test.data, dtype=torch.float32).div_(255)
y_test = torch.tensor(dset_test.targets, dtype=torch.int64)

In [7]:
x_test = torch.flatten(x_test,start_dim=1) #Flattening the 3 dimension image into a single dimension for easier calculation

In [8]:
x_test.shape

torch.Size([10000, 3072])

**Total Training and Testing Samples**

In [9]:
print(f"Total Training Samples: {x_train.shape[0]} \nTotal Test Samples: {x_test.shape[0]}")

Total Training Samples: 50000 
Total Test Samples: 10000


**Loss Function**

**SVM Loss with L2 regularization**

In [10]:
def svm_loss_function(W, X, y, reg):
  loss = 0.0
  dW = torch.zeros_like(W) # initialize the gradient as zero
  num_classes = W.shape[1]
  num_train = X.shape[0]
  loss = 0.0
  #SVM Loss Function
  scores = X.mm(W)
  correct_class_score = scores[range(num_train),y]
  margin = scores -  correct_class_score.view(-1,1) + 1
  margin[range(num_train),y] = 0
  mask = (margin > 0)
  loss = margin[mask].sum()
  mask_correct_y = torch.zeros_like(scores,dtype=torch.bool)
  mask_correct_y[range(num_train),y] = True
  margin[margin> 0] = 1
  margin[margin< 0] = 0
  margin[mask_correct_y] = torch.sum(margin, axis = 1)*-1
  dW = margin.T.mm(X).T
  loss /= num_train
  dW /= num_train
  loss += reg * torch.sum(W * W) # SVM Loss Function with L2 Regularization
  dW+=2*reg*W
  return loss, dW


**Batch Size Function**

In [11]:
def batch_size_func(X, y, num_train, batch_size):
  X_batch = None
  y_batch = None

  indices = torch.randint(num_train, (batch_size,))
  y_batch = y[indices]
  X_batch = X[indices]

  return X_batch, y_batch

**Linear Classifier Training Function with Stochastic Gradient Descent (SGD) Optimization Approach**

In [12]:
def linear_classifier_training_func(X, y, learning_rate=0.01,
                            reg=0.01, num_iters=1000, batch_size=64):
  num_train, dim = X.shape
  num_classes = torch.max(y) + 1
  W = 0.000001 * torch.randn(dim, num_classes, device=X.device, dtype=X.dtype)
  loss_history = []
  best_loss=float('inf')
  for it in range(num_iters):
    X_batch, y_batch = batch_size_func(X, y, num_train, batch_size)

    loss, grad = svm_loss_function(W, X_batch, y_batch, reg)
    loss_history.append(loss.item())
    if loss.item() < best_loss: # Computing and Saving the Best Weights Matrix
      best_W = W
      best_loss = loss.item()
    W -= learning_rate * grad    # Stochastic Gradient Descent (SGD) Optimization Approach
  return best_W, W, loss_history, best_loss

**Linear Classifier Prediction Function**

In [13]:
def linear_classifier_predict_func(W, X):
  y_pred = torch.zeros(X.shape[0], dtype=torch.int64)
  _, y_pred = X.mm(W).max(dim = 1)
  return y_pred


**Confusion Matrix Function**

In [14]:
def conf_matrix_values(y_test,y_pred,class_value):
  TN, FP, FN, TP = 0, 0, 0 , 0
  y_test_class=[1 if i==class_value else 0 for i in y_test]
  y_pred_class=[1 if i==class_value else 0 for i in y_pred]
  for i in range(len(y_test)):
    if y_test_class[i] == 0:
      if y_pred_class[i] == 0:
        TN+=1
      else:
        FP+=1
    if y_test_class[i] == 1:
      if y_pred_class[i] == 1:
        TP+=1
      else:
        FN+=1
  accuracy = (TP+TN)/(TP+FN+FP+TN)
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  #TP, FP, TN, FN
  return f"TP : {TP}, FP : {FP}, TN : {TN}, FN : {FN}\nAccuracy : {accuracy}\nPrecision : {precision}\nRecall : {recall}"

**Hyperparameter Tuning**

**Selecting the Best Values for Learning Rates and Regularization Strengths**

In [15]:
learning_rates = [0.000001,0.0001, 0.001, 0.005,0.01,0.05]
regularization_strengths = [0.001,0.5,1,3]

In [16]:
possible_combinations = list(itertools.product(learning_rates,regularization_strengths))
best_acc = 0
for i in range(len(possible_combinations)):
  l_r,r_s = possible_combinations[i]
  best_W, W, loss_history, best_loss = linear_classifier_training_func(X=x_train, y=y_train, learning_rate=l_r,reg=r_s, num_iters=10000, batch_size=64)
  y_pred = linear_classifier_predict_func(best_W,x_test)
  acc_score = accuracy_score(y_test,y_pred)
  if acc_score > best_acc:
    best_acc = acc_score
    best_comb = (l_r,r_s)

**Best Values for Learning Rates and Regularization Strengths after Hyperparameter Tuning**

In [17]:
print(f"Best Combination:\nLearning Rate : {best_comb[0]}\nRegularization Strength : {best_comb[1]}")

Best Combination:
Learning Rate : 0.0001
Regularization Strength : 0.001


In [18]:
l_r,r_s = best_comb

**Linear Classifier Training**

In [19]:
start = time.time()
best_W, W, loss_history, best_loss = linear_classifier_training_func(X = x_train, y = y_train, learning_rate=l_r,
                         reg=r_s, num_iters=10000, batch_size=64)
training_time = time.time()-start
print(f"Training Time Taken : {training_time}")

Training Time Taken : 48.804234981536865


**Linear Classifier Prediction on Testing Dataset**

In [20]:
start = time.time()
y_pred = linear_classifier_predict_func(best_W,x_test)
test_time = time.time() - start
print(f"Testing Time Taken : {test_time}")

Testing Time Taken : 0.20165014266967773


**Time Taken for Training and Test - Linear Classifier**

In [21]:
print(f"Training Time Taken - Linear Classifier : {training_time} secs")
print(f"Testing Time Taken - Linear Classifier : {test_time} secs")

Training Time Taken - Linear Classifier : 48.804234981536865 secs
Testing Time Taken - Linear Classifier : 0.20165014266967773 secs


**Best Weights (obtained after training)**

In [22]:
print("Best Weights (obtained after training):")
best_W

Best Weights (obtained after training):


tensor([[ 0.0008, -0.0017, -0.0079,  ..., -0.0026,  0.0188, -0.0002],
        [ 0.0049,  0.0094, -0.0157,  ..., -0.0105,  0.0094, -0.0087],
        [ 0.0081,  0.0023, -0.0133,  ..., -0.0103,  0.0098, -0.0126],
        ...,
        [-0.0026,  0.0045,  0.0022,  ..., -0.0007, -0.0002, -0.0111],
        [ 0.0026,  0.0079, -0.0084,  ..., -0.0008, -0.0098, -0.0091],
        [ 0.0141,  0.0057,  0.0026,  ...,  0.0032, -0.0033, -0.0009]])

**Loss value on Best Weights**

In [23]:
print("Loss value on Best Weights:", best_loss)

Loss value on Best Weights: 26.01335906982422


**Confusion Matrix Individual Results for 20 Labels - Linear Classifier**

In [24]:
for i in range(20):
  print(f"Class {i+1}: Class-{i+1} Label")
  print(conf_matrix_values(y_test,y_pred,i))
  print()

Class 1: Class-1 Label
TP : 26, FP : 66, TN : 9834, FN : 74
Accuracy : 0.986
Precision : 0.2826086956521739
Recall : 0.26

Class 2: Class-2 Label
TP : 12, FP : 89, TN : 9811, FN : 88
Accuracy : 0.9823
Precision : 0.1188118811881188
Recall : 0.12

Class 3: Class-3 Label
TP : 10, FP : 131, TN : 9769, FN : 90
Accuracy : 0.9779
Precision : 0.07092198581560284
Recall : 0.1

Class 4: Class-4 Label
TP : 11, FP : 251, TN : 9649, FN : 89
Accuracy : 0.966
Precision : 0.04198473282442748
Recall : 0.11

Class 5: Class-5 Label
TP : 0, FP : 2, TN : 9898, FN : 100
Accuracy : 0.9898
Precision : 0.0
Recall : 0.0

Class 6: Class-6 Label
TP : 15, FP : 184, TN : 9716, FN : 85
Accuracy : 0.9731
Precision : 0.07537688442211055
Recall : 0.15

Class 7: Class-7 Label
TP : 18, FP : 107, TN : 9793, FN : 82
Accuracy : 0.9811
Precision : 0.144
Recall : 0.18

Class 8: Class-8 Label
TP : 12, FP : 96, TN : 9804, FN : 88
Accuracy : 0.9816
Precision : 0.1111111111111111
Recall : 0.12

Class 9: Class-9 Label
TP : 8, FP 